In [44]:
!pip install scikit-learn -q

In [57]:
!pip install torchsummaryX wandb --quiet

In [113]:
!pip install tqdm --quiet

In [137]:
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torchsummaryX import summary
import wandb
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [70]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

MPS backend is available!


In [25]:
df = pd.read_csv("merged_output2.csv")

In [26]:
df = df.dropna()

In [ ]:
na_rows = df[df.isna().any(axis=1)]

In [29]:
na_rows

,Unnamed: 0_x,Timestamp,People Count,RGB,Pico,Target


In [46]:
df

,Unnamed: 0_x,Timestamp,People Count,RGB,Pico,Target
0,6206.0,2025-05-02 11:28:08.000,4.0,4.0,5.0,5.0
1,6205.0,2025-05-02 11:28:07.000,3.0,4.0,5.0,4.0
2,6204.0,2025-05-02 11:28:06.000,3.0,4.0,5.0,4.0
3,6203.0,2025-05-02 11:28:05.000,2.0,3.0,5.0,4.0
4,6202.0,2025-05-02 11:28:03.000,2.0,3.0,5.0,4.0
...,...,...,...,...,...,...
6008,10.0,2025-04-28 16:04:14.000,4.0,4.0,4.0,4.0
6010,8.0,2025-04-28 16:04:12.000,4.0,5.0,4.0,4.0
6011,7.0,2025-04-28 16:04:11.000,4.0,4.0,4.0,4.0
6012,6.0,2025-04-28 16:04:10.000,5.0,4.0,4.0,4.0


In [39]:
len(training_data)

5569

In [47]:
x = df.iloc[:, 2:5]

In [48]:
y = df.iloc[:, -1]
y

0       5.0
1       4.0
2       4.0
3       4.0
4       4.0
       ... 
6008    4.0
6010    4.0
6011    4.0
6012    4.0
6013    4.0
Name: Target, Length: 5569, dtype: float64

# NN MLP

In [101]:
train, temp = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)


In [102]:
test.iloc[:, 2:5]

,People Count,RGB,Pico
160,2.0,3.0,4.0
1356,3.0,5.0,4.0
770,2.0,3.0,5.0
5596,3.0,3.0,0.0
1235,3.0,6.0,4.0
...,...,...,...
1433,3.0,5.0,4.0
5995,4.0,3.0,4.0
3114,1.0,3.0,0.0
1125,3.0,4.0,4.0


In [103]:
class OccupancyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        return (torch.tensor(self.x.iloc[index].values, dtype=torch.float32),
                torch.tensor(self.y.iloc[index], dtype=torch.float32))


training = OccupancyDataset(train.iloc[:, 2:5],  train.iloc[:, -1])
validation = OccupancyDataset(val.iloc[:, 2:5],  val.iloc[:, -1])
test = OccupancyDataset(test.iloc[:, 2:5],  test.iloc[:, -1])

In [131]:
config = {
    'activations': 'GELU',
    'learning_rate': 0.001,
    'max_lr' : 0.007,
    'pct_start': 0.45,
    'optimizers': 'AdamW',
    'scheduler': 'OneCycleLR', #'ReduceLROnPlateau'
    'epochs': 20,
    'batch_size': 32,
    'step_size' : 2,
    'weight_initialization': 'kaiming_normal', # e.g kaiming_normal, kaiming_uniform, uniform, xavier_normal or xavier_uniform
    'drop_pct':0.07
 }

In [151]:
train_loader = torch.utils.data.DataLoader(
    dataset     = training,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
)


val_loader = torch.utils.data.DataLoader(
    dataset     = validation,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

In [152]:
all = []
for i, data in enumerate(val_loader):
    sensor_data, target = data
    all.append(target)
    print(len(sensor_data))
    break

32


In [153]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super().__init__()
        self.sequential = nn.ModuleList()

        #input layer
        self.sequential.append(nn.Linear(input_dim, hidden_dim))
        self.sequential.append(nn.ReLU())

        #hidden
        for i in range(layers):
            self.sequential.append(nn.Linear(hidden_dim, hidden_dim))
            self.sequential.append(nn.ReLU())

        #output layer
        self.sequential.append(nn.Linear(hidden_dim, output_dim))

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                if config["weight_initialization"] == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                elif config["weight_initialization"] == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                elif config["weight_initialization"] == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "uniform":
                    torch.nn.init.uniform_(m.weight)
                else:
                    raise ValueError("Invalid weight_initialization value")
                m.bias.data.fill_(0)
    def forward(self, x):
        for layer in self.sequential:
            x = layer(x)
        return y

In [154]:
model = MLP(3, 20, 1, 30).to(device)
summary(model, sensor_data.to(device))


----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                     [3, 20]             [32, 20]                 0.08                 0.00
1_ReLU                             -             [32, 20]                    -                    -
2_Linear                    [20, 20]             [32, 20]                 0.42                 0.00
3_ReLU                             -             [32, 20]                    -                    -
4_Linear                    [20, 20]             [32, 20]                 0.42                 0.00
5_ReLU                             -             [32, 20]                    -                    -
6_Linear                    [20, 20]             [32, 20]                 0.42                 0.00
7_ReLU                             -             [32, 20]                    -                    -

In [155]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
total_steps = config["epochs"]*len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                       max_lr = config['max_lr'], 
                                                       pct_start = config['pct_start'], 
                                                       total_steps = total_steps,
                                                       anneal_strategy = 'cos'
                                                       )
criterion = torch.nn.MSELoss()


In [127]:
wandb.login(

    key="f449e5715dbf82026aae85dffabb14116b5aa142"

)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/janbol/.netrc


True

In [161]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    tloss = 0.0
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, desc="Training")
    for i, (input, target) in enumerate(dataloader):
        optimizer.zero_grad()

        input = input.to(device)
        target = target.to(device)
        print(target.shape)

        logits = model(input)
        print(logits.shape)
        loss = criterion(input, target)
        tloss+=loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

        batch_bar.update()
    batch_bar.close()
    tloss /= len(train_loader)
    return tloss


In [ ]:
def eval(model, dataloader):
    model.eval()
    vloss = 0.0
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Validation')
    with torch.no_grad():
        for i, (input, target) in enumerate(dataloader):
            optimizer.zero_grad()

            input = input.to(device)
            target = target.to(device)
            

            logits = model(input)
            loss = criterion(logits, target)
            vloss+=loss.item()

            batch_bar.update()
            print("logits shape:", logits.shape)
            print("target shape:", target.shape)

    batch_bar.close()
    vloss /= len(val_loader)
    return vloss



In [125]:
wandb.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [133]:
run = wandb.init(
    project="autonomous_project",  # Specify your project
    config = config,
    name = "run1"
)

In [160]:
for epoch in range(config["epochs"]):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))
    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})





Epoch 1/20


torch.Size([32])
(5569,)


TypeError: 'int' object is not callable

In [142]:
target.shape

torch.Size([32])

In [143]:
sensor_data.shape

torch.Size([32, 3])